# ECS2XLS конвертор експорта трендов XML в XLSX,
## Назначение,
Преобразование  xml файла экспорта трендов ECS8 в EXCEL.,
## Использование,
 - загрузить XML файл в папку `./trends`;,
 - нажать кнопку "обновить список";,
 - выбрать в dropdown меню загруженный файл;,
 - нажать конпку \"конверитировать\";,
 - дождаться окончания работы скрипта;,
 - из папки `./trends` скачать файл с именем XML отчёта  и расширением xlsx.,
 

In [ ]:
import logging
import os
import time
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
from pathlib import Path
import ipywidgets as wg  # элементы управления для jupyter

# import numpy as np
import pandas as pd
from alive_progress import alive_bar, config_handler

In [ ]:
DEBUG = False
DATA = []
QNTY_TAGS = 0  # колличество тегов
QNTY_VALS = 0  # колличество значений
_PRG_DIR = Path("./").absolute()
_TRND_DIR = _PRG_DIR / "trends/"
_LOG_FILE = _PRG_DIR / "xml2xls.log"
_MAX_ROWS_EXL = 1000000  # макс. кол-во строк в Excel 1048000
_MAGADAN_UTC = 11  # Магаданское время +11 часов к UTC
log_format = f"%(asctime)s - %(levelname)s -(%(funcName)s(%(lineno)d) - %(message)s"
logging.basicConfig(
    format=log_format,
    level=logging.DEBUG if DEBUG else logging.INFO,
    # filename=_LOG_FILE,
)
logger = logging.getLogger()


In [ ]:
def get_trends_files_name(sortby="time", rev=True) -> list:
    """Получение списка XML файлов в папке Trends"""
    sort = {"time": os.path.getmtime, "size": os.path.getsize, "name": None}
    files = []
    for file in sorted(_TRND_DIR.glob("*.xml"), reverse=rev, key=sort[sortby]):
        files.append(file.name)
    return files

In [ ]:
def open_xml(xml_file_name: str):
    """"""
    tree = ET.parse(_TRND_DIR / xml_file_name)
    root = tree.getroot()
    return root
def get_trends_info(xml_root) -> dict:
    "Извлечение из xml информации по трендам"
    trends_info = {}
    date_format = "%Y-%m-%dT%H:%M:%S"
    # период выборки
    trends_info["start_interval"] = datetime.strptime(
        xml_root[1].text[:19], date_format
    )
    trends_info["end_interval"] = datetime.strptime(xml_root[0].text[:19], date_format)
    # указатель на теги в xml
    trends_info["xml_tags"] = xml_root[3]
    # колличество тегов
    trends_info["qty_tags"] = len(xml_root[3])
    return trends_info
def get_new_progbar(min=0, max=100):
    pg = wg.IntProgress(
        value=0,
        min=0,
        max=100,
        description="",
        bar_style="",  # 'success', 'info', 'warning', 'danger' or ''
        style={"bar_color": "maroon"},
        orientation="horizontal",
    )
    return pg
def get_tag_meta(tag):
    pass

In [ ]:
def make_list_for_df(enteries) -> list:
    """"""
    data = []
    with w_output_log:
        with alive_bar(len(enteries), force_tty=True, length=30) as bar:
            for enterie in enteries:
                # qnty_vals_l += 1
                # w_prgbar_step_cnt += 1
                str_time = enterie[1].text[:25].replace("Z", "")
                time_v = datetime.strptime(str_time, "%Y-%m-%dT%H:%M:%S.%f") + timedelta(
                    hours=_MAGADAN_UTC
                )  # enterie.find('{Fls.Core.Value.CI.Export.M1}Time').text
                value = enterie[2].text  
                # добавление строк в Pandas dataframe
                # df_tag.loc[len(df_tag)] = [time_v, value]
                data.append((time_v, value))
                bar()
    return data

In [ ]:
def convert_xml2xls(xml_file_name):
    """"""
    start_time = time.time()
    data = []  #
    data2 = []
    w_output_log.clear_output()
    file = _TRND_DIR / xml_file_name
    file_size = file.stat().st_size / 1048576 
    with w_output_log:
        print(f"\n========================")
        print(f"Открывается файл: {xml_file_name} Размер:  {round(file_size)} Мб")
        with alive_bar(1, force_tty=True, length=3 ) as bar:
        # открытие xml файла
            xml_root = open_xml(xml_file_name)
            bar()
        # получение информации о трендах
        trends_info = get_trends_info(xml_root)
        print(f"Открыт за {time.time() - start_time}")
        print(f"Интервал: {trends_info['start_interval']} - {trends_info['end_interval']}")
        print(f"Колличество тегов: {trends_info['qty_tags']}")
        
        print(f"\nФормируется датафрейм")
        s_time = time.time()
        qnty_tags = 0
        qnty_vals = 0
        qnty_vals_l = 0
        # Перебор тегов
        for tag in trends_info["xml_tags"]:  # xml_root[3]:
            qnty_tags += 1
            # if qnty_tags > 2: break
            # имя тега
            designation = tag.find("{Fls.Core.Value.CI.Export.M1}Designation").text
            unit = tag.find("{Fls.Core.Value.CI.Export.M1}Unit").text
            value_entries = []
            # date_format = '%Y-%m-%dT%H:%M:%S.%f'
            df_tag = pd.DataFrame(columns=[f"{designation}_dt", f"{designation}_value"])
            enteries = tag.find("{Fls.Core.Value.CI.Export.M1}valueEntries")
            print(f"\n-----------------------")
            print(f"Обрабатывается тег:{designation}; Количество значений: {len(enteries)}")
                # display(w_progbars[designation])
            s_time = time.time()
            qnty_vals_l = 0
            data2 = make_list_for_df(enteries)
            df_tag = pd.DataFrame(data2, columns=["dt", "value"])
            data2.clear()
            # df_tag = df_tag.iloc[:1048576] # макс. колл-во строк в EXEL <= 1048576
            data.append([designation, unit, df_tag])
            qnty_vals += qnty_vals_l
            print(
                    f"Обработан за:{time.time() - s_time} ; Тег/сек: {len(tag) / (time.time() - s_time)}"
                )
        print(f"\n========================")
        print(f"Парсинг закончен за время: {time.time() - start_time}")
        print(
            f"Обработано: Тегов {qnty_tags}; Значений: {qnty_vals}; Тег/сек: {qnty_vals / (time.time() - start_time)}"
        )
        print(f"\n\nНачало сохранения в XLSX {xml_file_name}")
        save_as_xlsx(data, xml_file_name + ".xlsx")
        print(f"\nExсel сохранён")
        print(f"\nВесь процесс занял времени:{time.time() - start_time}")

In [ ]:
def save_as_xlsx(dataframes: list, filename):
    "Сохраняет переданные datafaremes в xlsx"
    writer = pd.ExcelWriter(_TRND_DIR / filename, engine="xlsxwriter")
    workbook = writer.book
    #перебот тегов 
    with alive_bar(len(dataframes), force_tty=True, length=3 ) as bar:
        for tag in dataframes:
            df = tag[2]  #
            tag_name = tag[0]
            # колл-во строк в DF
            count_row = df.shape[0]
            # колличество вкладок excel 
            qty_sheets = count_row // _MAX_ROWS_EXL + 1 if (count_row > _MAX_ROWS_EXL) else 1
            logger.debug(f"Тег: {tag[0]}; count row: {count_row}; pages:{qty_sheets} ")
            for i in range(qty_sheets):
                logger.debug(f"Перебор листов i {i}")
                sheet_name = f"{tag_name} {i+1}"
                first_row = i * _MAX_ROWS_EXL
                end_row = (
                    ((i + 1) * _MAX_ROWS_EXL)
                    if count_row >= ((i + 1) * _MAX_ROWS_EXL)
                    else count_row
                )
                logger.debug(f"sheet_name: {sheet_name}; first_row: {first_row}; end row: {end_row}")
                df_page = df.iloc[first_row:end_row]
                with w_output_log:
                    print(f"Сохраняется лист: {sheet_name}; колл записей: {df_page.shape[0]};\n  дата от {df_page.iat[0,0]} до {df_page.iat[df_page.shape[0]-1,0]} \n")
                df_page.to_excel(writer, sheet_name=sheet_name, startrow=2, startcol=0, index=False)
                worksheet = writer.sheets[sheet_name]
                worksheet.write(0, 0, f"Значения тега {tag_name} за период c {df_page.iat[0,0]} по {df_page.iat[df_page.shape[0]-1,0]}")
            bar() 
    writer.close()

#### виджеты

In [ ]:
w_prgbar = wg.IntProgress(
    value=0,
    min=0,
    max=100,
    description="",
    bar_style="",  # 'success', 'info', 'warning', 'danger' or ''
    style={"bar_color": "maroon"},
    orientation="horizontal",
)
w_html1 = wg.HTML(
    value="""<H1>Конвертер XML to XLSX"</H1>
    """,
    placeholder="",
    description="",
)
w_html2 = wg.HTML(
    value="""<H2>Выберите файл для выгрузки или починки:</H2>\\
    <p>Файлы сохраняются в папку <b>mim_and_fp</b>, см. панель слева.</p>
    <p>Способ починки мимика описан в начале документа.</p>
    """,
    placeholder="",
    description="",
)
w_btn_refresh_flile_list = wg.Button(
    description="обновить  список файлов",
    disabled=False,
    button_style="",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click me",
    icon="check",  # (FontAwesome names without the `fa-` prefix)
)
w_btn_convert = wg.Button(
    description="конвертировать",
    disabled=False,
    button_style="",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click me",
    icon="check",  # (FontAwesome names without the `fa-` prefix)
)
w_xmlfiles = wg.Dropdown(options="Undef", description="XML файлы:")
def update_wg_droplist(b) -> None:
    w_xmlfiles.options = get_trends_files_name()
def covert(b) -> None:
    convert_xml2xls(w_xmlfiles.value)
update_wg_droplist("a")
w_btn_refresh_flile_list.on_click(update_wg_droplist)
w_btn_convert.on_click(covert)
w_output = wg.Output()
w_output_log = wg.Output()

## Конвертер XML to XLSX

In [ ]:
display(
    w_html1,
    wg.HBox([w_xmlfiles, w_btn_refresh_flile_list]),
    w_btn_convert,
    w_output_log,
)